In [ ]:
#SPIE 2022_Medical Imaging

In [ ]:
# !pip install --quiet --upgrade pip
# !pip install --quiet unet==0.7.7
# !pip install --quiet torchio==0.18.33
# !apt -qq install tree
#!pip install unet

In [ ]:
from __future__ import division, print_function, absolute_import

import math
import time
import sys
import cv2
import scipy
import os
import random
import itertools
import enum
import h5py
import zipfile
import pandas as pd
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import matplotlib.image as img
import nibabel as nb
import SimpleITK as sitk

from matplotlib import style
from pathlib import Path
from pathlib import Path
from functools import partial
from PIL import Image
from os import listdir
from os.path import join, basename, isdir
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from tqdm.notebook import trange, tqdm, tqdm_notebook
from scipy import stats
from scipy import ndimage
from matplotlib import style
from nibabel.testing import data_path
from ipywidgets import interact, fixed
from IPython import display
from tqdm.notebook import tqdm
from pathlib import Path



%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchio as tio
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchvision
import torchvision.utils
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import Compose, Lambda
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torchio.transforms import HistogramStandardization
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter


# import monai
# from monai.data import ImageDataset
# from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType
# from torch.optim import *


In [ ]:
torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
    
print('Using device ', device)

In [ ]:
print('TorchIO version:', tio.__version__)
print('Torch version:', tio.__version__)

In [ ]:
# Dataset

DATA_DIR = "D:/TSC/train_mix"

TRAIN_DIR_NAME = 'D:/TSC/train_mix/Train'

In [ ]:
os.listdir(TRAIN_DIR_NAME)

In [ ]:
# This is not entirely robust! The paths below DATA_DIR need to be like mine above.
source_files = []
#print("PATH:", DATA_DIR)
for current_file in Path(DATA_DIR).rglob('*.gz'):
    #print(current_file)
    source_files.append(str(current_file).split('\\')[-4:])
    #print(source_files)
source_files_df = pd.DataFrame(source_files)
#source_files_df.columns = ['a', 'b', 'c', 'd', 'set', 'class', 'filename']
source_files_df.columns = ['set','class', 'MRID', 'Sequences']

print(source_files_df.head())


In [ ]:
# This is not entirely robust! The paths below DATA_DIR need to be like mine above.
train_source_files = []
#print("PATH:", DATA_DIR)
for current_file in Path(TRAIN_DIR_NAME).rglob('*.gz'):
    #print(current_file)
    train_source_files.append(str(current_file).split('\\')[-4:])
    #print(source_files)
train_source_files_df = pd.DataFrame(train_source_files)
#source_files_df.columns = ['a', 'b', 'c', 'd', 'set', 'class', 'filename']
train_source_files_df.columns = ['set','class', 'MRID', 'Sequences']

print(train_source_files_df.head())

# #*********************************************


# Counting MRI sequences

In [ ]:
#Checking the number of MRI sequesnces/ what kind of MRI sequesces we want!

#effective_sequences = source_files_df.loc[(source_files_df['Sequences'] == 'FSPGR BRAVO.nii.gz')]
#effective_sequences = source_files_df.loc[(source_files_df['Sequences'] == 'AX T1 FLAIR.nii.gz')]
effective_sequences_TSC = train_source_files_df.loc[(train_source_files_df['Sequences'] == 'MR AX T2 FLAIR.nii.gz')]
effective_sequences_Normal = train_source_files_df.loc[(train_source_files_df['Sequences'] == 'AX T2 FLAIR.nii.gz')]
train_patient_ids = train_source_files_df.MRID.unique()    

In [ ]:
print("TSC Seq")
print(train_source_files_df.loc[(train_source_files_df['Sequences'] == 'MR AX T2 FLAIR.nii.gz')].nunique())

In [ ]:
print("Normal Seq")
print(train_source_files_df.loc[(train_source_files_df['Sequences'] == 'AX T2 FLAIR.nii.gz')].nunique())

# Train

In [ ]:
#Checking the number of MRI sequesnces/ what kind of MRI sequesces we want!

#effective_sequences = source_files_df.loc[(source_files_df['Sequences'] == 'FSPGR BRAVO.nii.gz')]
#effective_sequences = source_files_df.loc[(source_files_df['Sequences'] == 'AX T1 FLAIR.nii.gz')]
effective_sequences_TSC = train_source_files_df.loc[(train_source_files_df['Sequences'] == 'MR AX T2 FLAIR.nii.gz')]
effective_sequences_Normal = train_source_files_df.loc[(train_source_files_df['Sequences'] == 'AX T2 FLAIR.nii.gz')] 
    

import os
from tqdm import tqdm
#  read all train seqs 
Train_seqs=[]
train_labels=[]
train_subjects = []

classes={"TSC":1, "Normal":0}

train_patient_id=np.array(effective_sequences_TSC["MRID"])
train_seq_name=np.array(effective_sequences_TSC["Sequences"])


for i in tqdm(range(len(effective_sequences_TSC["Sequences"]))):    
    train_subjects.append(tio.Subject(
        mri=tio.ScalarImage(os.path.join(r"D:/TSC/train_mix/Train/TSC",train_patient_id[i],train_seq_name[i])),
        label=classes["TSC"],
        ))
    
############################################## Normal ########################################################
Ntrain_patient_id=np.array(effective_sequences_Normal["MRID"])
Ntrain_seq_name=np.array(effective_sequences_Normal["Sequences"])
                                        

for i in tqdm(range(len(effective_sequences_Normal["Sequences"]))):
    train_subjects.append(tio.Subject(
        mri=tio.ScalarImage(os.path.join(r"D:/TSC/train_mix/Train/Normal",Ntrain_patient_id[i],Ntrain_seq_name[i])),
        label=classes["Normal"],
        ))


In [ ]:
print('Dataset size:', len(train_subjects), 'subjects')

In [ ]:
one_subject = train_subjects[10]
one_subject.plot()

In [ ]:
print(one_subject)
print(one_subject.mri)

# Define transforms

In [ ]:
## Only augment training data

image_out_size = (200,200,100)

train_transforms = tio.Compose([
    tio.ToCanonical(),
    tio.Resample(2), # Isotropic (2,2,2)mm
    tio.CropOrPad(image_out_size),
#   tio.RandomMotion(p=0.2),
#   tio.RandomBiasField(p=0.3),
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),
    tio.RandomNoise(p=0.15),
    tio.RandomBlur(p=0.2),
    tio.RandomFlip(),
    # tio.Lambda(lambda x: patchSample(x))
    tio.OneOf({
        #tio.RandomAffine(): 0.4,
        tio.RandomElasticDeformation(): 0.1,
    }),
    tio.OneHot()])

val_transforms = tio.Compose([
        tio.ToCanonical(),
        tio.Resample(2),
        tio.CropOrPad(image_out_size),
        #tio.HistogramStandardization({'mri': landmarks}),
        tio.ZNormalization(masking_method=tio.ZNormalization.mean),
        tio.OneHot()]) 

#Splitting dataset to train, Valid and Test
idx = np.random.permutation(len(train_subjects))

subjTr =[]
subjVa =[]
subjTs =[]
for c , i in enumerate(idx):
    if c < int(0.7*len(train_subjects)):
        subjTr.append(train_subjects[i])
    elif c>= int(0.7*len(train_subjects)) and  c< int(0.9*len(train_subjects)):
        subjVa.append(train_subjects[i])
    else:
        subjTs.append(train_subjects[i])
            

# https://torchio.readthedocs.io/data/dataset.html

dataTr = tio.SubjectsDataset(subjTr, transform=train_transforms)
dataVa = tio.SubjectsDataset(subjVa, transform=val_transforms)

In [ ]:
print(len(subjTr), len(subjVa), len(subjTs))

# Create data loaders

In [ ]:
batch_size = 4
dataloaders = {
  'train': DataLoader(dataTr, batch_size=batch_size, shuffle=True, num_workers=4),
  'val': DataLoader(dataVa, batch_size=batch_size, shuffle=True, num_workers=4)
}

# 3D Model

#### 3D CNN

In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv3d(in_channels, out_channels, 3, padding=1),
        nn.InstanceNorm3d(out_channels),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv3d(out_channels, out_channels, 3, padding=1),
        nn.InstanceNorm3d(in_channels),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Dropout(0.25)
    )

class Encoder(nn.Module):

    def __init__(self, n_class = 1):
        super().__init__()
                
        self.dconv_down1 = double_conv(1, 16)
        self.dconv_down2 = double_conv(16, 32)
        self.dconv_down3 = double_conv(32, 64)
        self.dconv_down4 = double_conv(64, 128)
        #self.dconv_down5 = double_conv(128, 256)      
        self.avgpool = nn.AdaptiveAvgPool3d((1,1,1))       
        self.fc = nn.Linear(128, 1)

        self.maxpool = nn.MaxPool3d(2)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        conv4 = self.dconv_down4(x)
        x1 = self.maxpool(conv4)

        avgpool = self.avgpool(x1)
        avgpool = avgpool.view(avgpool.size(0), -1)
        outC = self.fc(avgpool)
        
        
        return torch.sigmoid(outC)
    
model = Encoder(1)

In [ ]:
print(model)

#### DenseNet

In [ ]:
# model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)

In [ ]:
# print(model)

### Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

from torchsummary import summary
print(summary(model, input_size=(1, 200, 200, 100)))

#### Layer Weight Shape

In [ ]:
# model.dconv_down4

In [ ]:
#model.dconv_down1.weight

In [ ]:
#model.fc.weight.shape

In [ ]:
#model.outC.weight.shape

In [ ]:
#print("Parameters: ", list(Encoder.parameters()))

In [ ]:
# for parm in model.parameters():
#     print(parm.shape)

In [ ]:
# for name, parm in model.named_parameters():
#     print(name, '\t\t' ,  parm.shape)

### Definition of hyperparameters

In [ ]:
#Definition of hyperparameters

num_epochs = 5


# Cross Entropy Loss 
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()

# Optimizer
learning_rate = 0.001
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
optimizer= torch.optim.RMSprop(model.parameters(), lr=learning_rate, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)


In [ ]:
import gc
device = torch.device('cuda')

model.to(device)
def train_model(model, optimizer, save_path, num_epochs=num_epochs):
    best_loss = 1e10

    epoch_Losses = {k:[] for k in ['train', 'val']}
    epoch_Accs = {k:[] for k in ['train', 'val']}
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #for i in tgdm(range(0,(Train_ds, batch_size))
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                # gc.collect()
                # torch.cuda.empty_cache()

                model.eval()   # Set model to evaluate mode

            Losses, correct, total = 0.0, 0.0, 0.0
            for i, batch in enumerate(dataloaders[phase]):
                gc.collect()
                torch.cuda.empty_cache()

                batch_images = batch['mri']['data'].to(device=device, dtype=torch.float)
                batch_labels = batch['label'].to(device=device, dtype=torch.float)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase=='train'):

                    # backward + optimize only if in training phase
                    if phase == 'train':

                        batch_labels_pred = model(batch_images)  # Pass batch
                        loss = criterion(batch_labels_pred[:, 0], batch_labels)
                        loss.backward()    # Calculate Gradient
                        optimizer.step()   # Update Weights
                        
                        l2_lambda = 0.001
#                         l2_norm = sum(p.pow(2.0).sum()
#                         for p in model.parameters())
                        loss = loss + l2_lambda #* l2_norm
    
                    else:
                        batch_labels_pred = model(batch_images)  # Pass batch
                        loss = criterion(batch_labels_pred[:, 0], batch_labels)
            

                # statistics
                Losses += loss
                pred=batch_labels_pred
                batch_labels_pred[pred>0.5]=1
                batch_labels_pred[pred<=0.5]=0
#                 _, predicted = torch.max(batch_labels_pred.data, 1)
                predicted = batch_labels_pred[:, 0] #.clone()
                total += batch_labels.size(0)
                #print(predicted)
                #print(batch_labels)
                #print('Current: ', torch.sum(predicted==batch_labels))
                correct += predicted.eq(batch_labels.data).sum() # (predicted == batch_labels).sum().item()
            #print(correct, total)
                

            epoch_acc = 100 * correct / total
            epoch_loss = Losses/len(dataloaders[phase])
            
            epoch_Losses[phase].append(epoch_loss.cpu().data.numpy())
            epoch_Accs[phase].append(epoch_acc.cpu().data.numpy())
            
            print("Epoch: %d/%d (%s) Loss: %f\tAcc: %f" % (epoch, num_epochs, phase, epoch_loss, epoch_acc))
            #print("Epoch: %d/%d (%s) Loss: %f" % (epoch, num_epochs, phase, epoch_loss))

            # save the model weights
            if phase == 'val':
                if epoch_loss < best_loss:
                    print(f"saving best model to {save_path}")
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), save_path)
        
        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    print('Best val loss: {:4f}'.format(best_loss))
    #print('Best val loss: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(torch.load(save_path))
    
    return model, epoch_Losses, epoch_Accs

In [ ]:
# for i, batch in enumerate(dataloaders['train']):
#   print(batch['mri']['data'].shape)
#   print(batch['label'])

In [ ]:
def test_model(model, save_path, phase='test'):
    
    model.load_state_dict(torch.load(save_path))
    
    since = time.time()

    model.eval()   # Set model to evaluate mode

    predictions = np.zeros((0)) 
    correct = total = 0
    for i, batch in enumerate(dataloaders[phase]):
        batch_images = batch['mri']['data'].to(device=device, dtype=torch.float)
        batch_labels = batch['label'].to(device=device, dtype=torch.float)

        # zero the parameter gradients
        batch_labels_pred = model(batch_images)  # Pass batch
        loss = criterion(batch_labels_pred, batch_labels)
        _, predicted = torch.max(batch_labels_pred.data, 1)
        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()
        #predictions = np.concatenate([predictions, predicted.cpu().data.numpy()], 0)
    time_elapsed = time.time() - since    
    epoch_acc = 100 * correct / total
    
    print(predictions.shape)
            
    print("Test tAcc: %f" % (epoch_acc))    
    print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return predictions

In [ ]:
#!mkdir pytorch_model

In [ ]:
save_path = './pytorch_model/model.pth'
model, Loss, Acc = train_model(model, optimizer, save_path, num_epochs=100)

In [ ]:
for key in ['train', 'val']:
    plt.plot(Loss[key], label=key+' Loss')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Classification Loss')
    plt.title('Loss Plot')
plt.show()    

for key in ['train', 'val']:
    plt.plot(Acc[key], label=key+' Acc')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Classification Accuracy')
    plt.title('Accuracy Plot')

# Confusion Matrix

In [ ]:
# Cinfusion Matrix

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sn
import pandas as pd

y_pred = []
y_true = []

# iterate over test data
for inputs, labels in test_loader:
        output = model(inputs.cuda()) # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output) # Save Prediction
        
        labels = labels.data.cpu().numpy()
        y_true.extend(labels) # Save Truth

# constant for classes
classes = ['Normal', 'TSC']

# Build confusion matrix
cf_matrix = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
plt.ylabel('True label', fontsize=12, color='darkblue') # fontweight='bold'
plt.xlabel('Predicted label\naccuracy={:0.2f}; misclass={:0.2f}', fontsize=12, color='darkblue')
sn.heatmap(df_cm, annot=True)
plt.savefig('output.png')
print(classification_report(y_true, y_pred))

# Statistical Result

In [ ]:
# Overall Statistics and  Class Statistics metrices
from pycm import *

cm = ConfusionMatrix(y_true, y_pred)
cm.table
print(cm)

In [ ]:
# ROC curve

from sklearn.metrics import roc_curve, auc


nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y, pred)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='3D CNN (auc = %0.3f)' % auc_keras)

plt.title("ROC Curve")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
print('Area under ROC curve : ', roc_auc_score(y, pred) *100 )